## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 


<font color='red'> Все импорты должны быть в отдельной ячейке сверху
</font>

In [308]:
import pandas as pd
data = pd.read_csv('data.csv', delimiter=',')
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [309]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


- Вывод

Перед нами список, состоящий из 12 столбцов и тысяч строк. <font color='red'> Скольки тысяч?
</font>

В строках встречаются различные типы данных, целые числа, вещественные и класс object. <font color='red'>Не класс object, а строки</font>


В столбце общий трудовой стаж: days employee  встречаются отрицательные значения, чего быть не должно

В столбце education значения забиты в разном регистре, что помешает дальнейшему анализу.

А в столбце purpose сразу бросается в глаза разичная формулировка целей: "приоберетние автомобиля" и "покупка автообиля" - это об одном и том же, поэтому необходимо найти все формлировки и привести их к одной общей.  

Так же с помощью метода .info(), я определила, что есть пропущенные значения в столбце с индексом 1 и 10.<font color='red'>Лучше указывать названия столбцов, а не их индексы</font>


### Шаг 2. Предобработка данных


Исследовала столбцы на нахождение данных "с ошибкой"

<font color='red'>Не связно, улучши повествование</font>
1. Привести все данные к нижнему регистру
2. Проверить по столбцу на нахождение необычных данных 
3. Исправить данные 
4. Создать  новую таблицу таблицу <font color='red'>Новую таблицу таблицу?</font>

Так как в data['days_employed'] преобладающее большинство значений записаны со знаком минус, а отрицательного стажа работы быть не должно, нам следует изменить значения на положительные. 


А в столбце data['education'] привести все значения к одному регистру.

## Шаг 2.1 Привести данные к нижнему регистру

In [380]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,credit_purpose,age_group
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,взрослые
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,взрослые
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,взрослые
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,взрослые
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,взрослые


In [311]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower() 
data['purpose'] = data['purpose'].str.lower()


<font color='red'>Ты не выровняла регистр в столбцах
    
Нужно использовать присваивание, а не просто вызов метода
</font>

<font color='red'> Лучше писать "Столбцы", а не сириусы
</font>

## Шаг 2. 2 Изучить сириусы education_id, family_status_id

In [312]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

<font color='red'> зачем тут метод head 5 раз подряд?
</font>

In [313]:

data[data['education_id'] == 0].head() #высшее
data[data['education_id'] == 1].head() #среднее
data[data['education_id'] == 2].head() #неоконч. высшее
data[data['education_id'] == 3].head() #начальное
data[data['education_id'] == 4].head() #ученая степень

#столбец с индексами пригодится нам для составления сводной таблицы и сравнения в конце исследования 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2963,0,337584.815560,69,ученая степень,4,женат / замужем,0,m,пенсионер,0,98752.495442,покупка жилой недвижимости
4170,0,-409.200149,45,ученая степень,4,не женат / не замужем,4,m,сотрудник,0,198570.757322,операции с коммерческой недвижимостью
6551,0,-5352.038180,58,ученая степень,4,женат / замужем,0,m,сотрудник,0,268411.214536,заняться высшим образованием
12021,3,-5968.075884,36,ученая степень,4,женат / замужем,0,f,госслужащий,0,111392.231107,покупка жилья
12786,0,376276.219531,62,ученая степень,4,женат / замужем,0,f,пенсионер,0,255425.196556,покупка жилой недвижимости


In [314]:
data_family_status_id = data['family_status_id'].value_counts()
data_family_status_id


0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

<font color='red'> тот же вопрос
</font>

In [315]:
data[data['family_status_id'] == 0].head() #замужем
data[data['family_status_id'] == 1].head() #гражданский брак
data[data['family_status_id'] == 2].head() #вдовец/ вдова
data[data['family_status_id'] == 3].head() #развод
data[data['family_status_id'] == 4].head() #холост
#столбец с индексами пригодится нам для составления сводной таблицы и сравнения в конце исследования 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
24,1,338551.952911,57,среднее,1,не женат / не замужем,4,f,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
29,0,NaN,63,среднее,1,не женат / не замужем,4,f,пенсионер,0,NaN,строительство жилой недвижимости
36,0,-176.216688,33,среднее,1,не женат / не замужем,4,m,сотрудник,0,138830.650734,автомобили
38,0,-597.881827,25,высшее,0,не женат / не замужем,4,m,сотрудник,1,192247.298639,образование
40,0,-1030.362606,30,среднее,1,не женат / не замужем,4,m,компаньон,0,142865.579146,покупка недвижимости


## Шаг 2.3 Исправить пропущенные значения data['days_employed']

In [316]:
data['days_employed'].value_counts()

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64

In [317]:
a = 0
for i in data['days_employed']:
    if i < 0:
        a += 1
#мы посчитали количество отрицательных значений с помощью цикла. 

In [318]:
a 
#Сохранили значение в переменной,
#затем посчитали отношение количества отрицательных значений к количеству всех фреймов в сериусе.

15906

In [319]:
lenght_not_null_days_emp = len(data) - len(data[data['days_employed'].isna()])
percent_plus_days_emp = round(a / lenght_not_null_days_emp * 100, 2) 
#процентное отношение отрицательных значений к количеству заполненных фреймов в сериусе
percent_plus_days_emp

82.2

<font color='red'> Процент нужно было считать ко всем данным, а не положительным, ведь мы хотим посмотреть какой процент отрицательные значения занимают от всех значений в датаферйме
</font>

Вывод: 
Так как процент достаточно большой, я задумалась, может ли это быть ошибкой (человеческий фактор) или значения не верны и стоит их просто удалить. 

Кредитный скоринг - система оценки, основанная на численных статистических методах. Стаж работы важный критерий в оценке кредитоспособности лица, удалить эти значения без разрешения банка я не могу, потому воспользуюсь методом замены отрицательных значений на положительные


In [320]:
#len(data.query('days_employed < 0 '))

Замена циклу

In [321]:
len(data[data['days_employed'].isna()])

2174

In [322]:
data['days_employed'] = abs(data['days_employed'])

In [323]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь стоит решить, что делать с пропущенными значениями, которых 2174! Найдем их процентное соотноение

In [324]:
round(len(data[data['days_employed'].isna()]) / len(data) * 100, 2)

10.1

10 % это немного, я скорее отнесу это к ошибке технического характера и заменю данные на средний стаж работы. Так как удаление данных приведет к их потере и, возможно, к ошибкам в выводах


In [325]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())

<font color='red'> 
Раз уж ты решила заполнить все просто средним, без разделения по категориям, то
здесь лучше использовать медиану, а не среднее, т.к. оно очень сильно подвержено выбросам в данных
</font>

Мы заменили пропущенные значения в сериусе data['days_employed']

## Шаг 2.4 Исследовать data['dob_years'] на наличие ошибок

In [326]:
data['dob_years'].value_counts()
len(data.query('dob_years < 18 '))
#заемщик кредита в банке не может быть младше 18 лет, здесь ошибка

101

In [327]:
round(len(data.query('dob_years < 18 ')) / len(data) * 100, 2)

0.47

In [328]:
data = data[data['dob_years'] >= 18]
len(data.query('dob_years < 18'))




0

In [329]:
#Во время исследования столбца dob_years было выявлено необычное явление
#когда люди 19 и чуть больше лет отнесены в столбце "income_type" к пенсионерам

В процентном соотношении получается меньше 1% от всех значений датафрейма, было принято решение удалить данные

<font color='red'> а где мне увидеть расчеты и удаление этих данных? Эти операции дожны быть здесь
</font>

<font color='green'> все расчеты представлены в шаге 2.4 и ничего лишнего
    

## Шаг 2.5 Исследовать data['children'] на ошибки в данных

In [330]:
data['children'].value_counts()
#здесь следует заменить значения -1 и 20 на 1 и 20 соответственно, 
#скорее всего это опечатки

 0     14080
 1      4802
 2      2042
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [331]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
data['children'].value_counts()


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


0    14080
1     4849
2     2117
3      328
4       41
5        9
Name: children, dtype: int64

In [332]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21424 non-null  int64  
 1   days_employed     21424 non-null  float64
 2   dob_years         21424 non-null  int64  
 3   education         21424 non-null  object 
 4   education_id      21424 non-null  int64  
 5   family_status     21424 non-null  object 
 6   family_status_id  21424 non-null  int64  
 7   gender            21424 non-null  object 
 8   income_type       21424 non-null  object 
 9   debt              21424 non-null  int64  
 10  total_income      19260 non-null  float64
 11  purpose           21424 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Таким образом мы:
нашли все пропущенные значения с помощью метода .isnull()и артефакты при помощи .value_counts() 
заменили пропуски при помощи .loc и удалили пропуски при помощи условия '>='. А так же привели к ниэнему регистру все сериусы с типом данных object


## Шаг 2.6 Обработка пропусков в data['total_income']
Исследовать столбцы data['gender'] data['debt']


Мы заменили пропущенные значения в сериусе data['days_employed']. Так же пропущенные значения встречаются в сериусе total_income в таком же количестве у тех же людей, скорее это тоже ошибка технического характера, так как сферв занятости семейный статус, возраст клиентов разный и нет видимых причин, почему пропуски могли бы быть оставлены специально. Проведем с ним такую же операцию. И сделаем проверку таблицы методом isna() на наличие пропусков

In [333]:
data['total_income'] = data['total_income'].fillna(data['total_income'].mean())

<ipython-input-333-614e5baba0f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_income'] = data['total_income'].fillna(data['total_income'].mean())


<font color='red'> Тут также нужно либо заменять на среднее согласно категориям, либо использовать медиану, но лучше по категориям
+ я не вижу, где ты посмотрела, что пропуски в столбце total_income именно у тех же людей, у которых пропуски в days_employed

</font>

In [334]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [335]:
data['total_income'].value_counts()

167465.224095    2164
89922.645314        1
133912.272223       1
184315.121979       1
182036.676828       1
                 ... 
115080.782380       1
84896.781597        1
153838.839212       1
177314.197183       1
150014.128510       1
Name: total_income, Length: 19261, dtype: int64


<font color='red'> Тебе не кажется, что среднее значение в 167465 слишком сильно отличается от правды? еще раз напоминание про медиану
</font>


In [336]:
data['gender'].value_counts()
#мы обнаружили один пропуск в граве "пол", так как он всего 1, не повлияет на резултат исследования

f      14164
m       7259
xna        1
Name: gender, dtype: int64

Пропущенных значений больше нет. Мы заполнили их средним значением по соответвующему столбцу, так как удаление 1/10 данных может привести к ошибке в выводе.


После обработки пропусков нам следует проверить другие столбцы на наличие артефактов. 

In [337]:
data['debt'].value_counts()

0    19691
1     1733
Name: debt, dtype: int64

Вывод: мы исследовали данные на наличие пропусков и ошибок. Таким образом, были выявлены и исправлены ошибки в столбцах: children,days _employed, dob_years, total_income. Данные в таблице были приведены к нижнему регистру для удобства дальнейшей работы.	

## Шаг 2.7

### Замена типа данных days _employed, total_income

In [338]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21424 non-null  int64  
 1   days_employed     21424 non-null  float64
 2   dob_years         21424 non-null  int64  
 3   education         21424 non-null  object 
 4   education_id      21424 non-null  int64  
 5   family_status     21424 non-null  object 
 6   family_status_id  21424 non-null  int64  
 7   gender            21424 non-null  object 
 8   income_type       21424 non-null  object 
 9   debt              21424 non-null  int64  
 10  total_income      21424 non-null  float64
 11  purpose           21424 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [339]:
#заменим тип float на int
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21424 non-null  int64 
 1   days_employed     21424 non-null  int64 
 2   dob_years         21424 non-null  int64 
 3   education         21424 non-null  object
 4   education_id      21424 non-null  int64 
 5   family_status     21424 non-null  object
 6   family_status_id  21424 non-null  int64 
 7   gender            21424 non-null  object
 8   income_type       21424 non-null  object
 9   debt              21424 non-null  int64 
 10  total_income      21424 non-null  int64 
 11  purpose           21424 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


<ipython-input-339-16fd6ed834c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['days_employed'] = data['days_employed'].astype(int)
<ipython-input-339-16fd6ed834c2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_income'] = data['total_income'].astype(int)


Вывод: изменили тип данных на целочисленный методом astype(int)

## Шаг 2.8

### Обработка дубликатов


In [340]:
data.duplicated().sum() #мы нашли 54 дубликата
#data

71

In [341]:
data = data.drop_duplicates().reset_index(drop = True)

In [342]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21353 entries, 0 to 21352
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21353 non-null  int64 
 1   days_employed     21353 non-null  int64 
 2   dob_years         21353 non-null  int64 
 3   education         21353 non-null  object
 4   education_id      21353 non-null  int64 
 5   family_status     21353 non-null  object
 6   family_status_id  21353 non-null  int64 
 7   gender            21353 non-null  object
 8   income_type       21353 non-null  object
 9   debt              21353 non-null  int64 
 10  total_income      21353 non-null  int64 
 11  purpose           21353 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Вывод:
C помощью метода .duplicates() и .sum() мы проверили весь датафрейм на наличие полных дубликатов, 
метод value_counts() не подойдет, так как исследует значения только в одной колонке. Вызов info() проверил, удалились ли дубликаты 

## Шаг 2.9

### Лемматизация

In [343]:
from pymystem3 import Mystem

In [344]:

m = Mystem()
lemmas = m.lemmatize('арбузный')
#lemmas = m.lemmatize(data.loc[data['purpose'][0:37]]lemmas = m.lemmatize(data.loc[data['purpose'][0:37]]')
#from collections import Counter
#print(Counter(lemmas))


In [345]:
data['lemma'] = data['purpose'].apply(lambda x: m.lemmatize(x))

In [346]:
data.head()



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [347]:
def listmerge(lstlst):
    all=[]
    for lst in lstlst:
        all.extend(lst)
    return all



In [348]:

lemmas_list = listmerge(data['lemma'])

In [349]:
from collections import Counter

In [350]:
Counter(lemmas_list).most_common()



[(' ', 33435),
 ('\n', 21353),
 ('недвижимость', 6328),
 ('покупка', 5870),
 ('жилье', 4436),
 ('автомобиль', 4284),
 ('образование', 3995),
 ('с', 2904),
 ('операция', 2593),
 ('свадьба', 2310),
 ('свой', 2223),
 ('на', 2210),
 ('строительство', 1873),
 ('высокий', 1366),
 ('получение', 1309),
 ('коммерческий', 1306),
 ('для', 1286),
 ('жилой', 1224),
 ('сделка', 938),
 ('дополнительный', 902),
 ('заниматься', 900),
 ('проведение', 764),
 ('сыграть', 760),
 ('сдача', 649),
 ('семья', 637),
 ('собственный', 633),
 ('со', 627),
 ('ремонт', 605),
 ('подержанный', 484),
 ('подержать', 478),
 ('приобретение', 459),
 ('профильный', 435)]

In [351]:
#for i in range(len(data['lemma'])):
#part = data[i][1]/emojixpress_total data[i].append(part)

In [352]:
def listmerge2(data):
    credit_purpose = []
    words = list(data['lemma'])
    for i in words:
        if 'автомобиль' in words:
            return 'автомобиль'
        if 'жилье' in words:
            return 'недвижимость'
        if 'недвижимость' in words:
            return 'недвижимость'
        if 'образование' in words:
            return 'образование'
        if 'свадьба'in words:
            return 'свадьба'
data['credit_purpose'] = data.apply(listmerge2, axis = 1)
#1-автомобиль
#2-недвижимость
#3-образование
#4-свадьба

In [353]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
lemma               0
credit_purpose      0
dtype: int64

Вывод: Провели лемматизацию датафрейма. 1 Шаг - вызвать словарь. 2- создан новый столбец lemma, который равен  data['purpose']? к которому применена функция lambda, которая выполняет лемматизацию значений датафрейма purpose.
Затем мы применили функцию listmerge к столбцу с леммами, вызвали словарь Counter и смогли посмотреть на самые most_common() леммы в датафрейме.
На основе этого мне удалось создать цикл, где при помощи условия я смогла привести все цели кредита к общим, таким образом, можно переходить к категоризации данных

## Шаг 2.10

### Категоризация


In [354]:
data.sort_values(by = 'dob_years')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,credit_purpose
2711,0,115,19,среднее,1,не женат / не замужем,4,f,компаньон,0,131967,операции с коммерческой недвижимостью,"[операция, , с, , коммерческий, , недвижимо...",недвижимость
12934,0,695,19,неоконченное высшее,2,не женат / не замужем,4,f,компаньон,0,165269,автомобили,"[автомобиль, \n]",автомобиль
10173,0,793,19,среднее,1,женат / замужем,0,f,сотрудник,0,131308,ремонт жилью,"[ремонт, , жилье, \n]",недвижимость
4031,0,66914,19,среднее,1,гражданский брак,1,m,сотрудник,0,167465,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
20069,0,509,19,неоконченное высшее,2,не женат / не замужем,4,f,госслужащий,0,75787,покупка коммерческой недвижимости,"[покупка, , коммерческий, , недвижимость, \n]",недвижимость
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3440,0,344623,74,среднее,1,женат / замужем,0,m,пенсионер,0,54754,операции со своей недвижимостью,"[операция, , со, , свой, , недвижимость, \n]",недвижимость
4870,0,341528,74,высшее,0,женат / замужем,0,f,пенсионер,0,134935,покупка своего жилья,"[покупка, , свой, , жилье, \n]",недвижимость
12235,0,1729,74,среднее,1,женат / замужем,0,m,компаньон,0,124830,покупка жилой недвижимости,"[покупка, , жилой, , недвижимость, \n]",недвижимость
11457,0,6682,74,среднее,1,гражданский брак,1,f,сотрудник,0,98945,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


In [355]:

data_retiree = data[data['income_type'] == 'пенсионер'].sort_values(by = 'dob_years')

классифицируем таблицу по возрастным категориям , так как при сравнении данных возникло сомнение, что данные в income_type корректные

In [356]:
def age_group(dob_years):
        if dob_years < 60:
                return 'взрослые'            
        return 'пенсионеры' 
data['age_group'] = data['dob_years'].apply(age_group)
data['age_group']


0          взрослые
1          взрослые
2          взрослые
3          взрослые
4          взрослые
            ...    
21348      взрослые
21349    пенсионеры
21350      взрослые
21351      взрослые
21352      взрослые
Name: age_group, Length: 21353, dtype: object

In [357]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,credit_purpose,age_group
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,взрослые
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,взрослые
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,взрослые
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,взрослые
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,взрослые


Отсортирум таблицу, сохранив в переменной data_sort необходимые нам фреймы для анализа

In [358]:
data_sort = data[['children', 'education_id', 'family_status_id', 'debt', 'total_income', 'credit_purpose', 'age_group']]

In [359]:
#data_sort = data[data['debt'] == 1]

In [360]:
data_sort.head()


,children,education_id,family_status_id,debt,total_income,credit_purpose,age_group
0,1,0,0,0,253875,недвижимость,взрослые
1,1,1,0,0,112080,автомобиль,взрослые
2,0,1,0,0,145885,недвижимость,взрослые
3,3,1,0,0,267628,образование,взрослые
4,0,1,1,0,158616,свадьба,взрослые


Приступаем к анализу данных. Для ответа на поставленные вопросы будет удобно использовать сводные таблицы. 

In [361]:
data_family = data_sort.pivot_table(index=['debt'], columns='family_status_id', values='children', aggfunc='count')
data_family.reset_index(drop=True)
#мы создали сводную таблицу
#затем посчитаем данные в процентном соотношении

family_status_id,0,1,2,3,4
0,11363,3744,892,1100,2521
1,927,386,62,85,273


In [362]:
data_family.loc['total'] = data_family.loc[0] + data_family.loc[1]
data_family
#посчитали общее количество данных по каждому статусу

family_status_id,0,1,2,3,4
debt,,,,,
0,11363,3744,892,1100,2521
1,927,386,62,85,273
total,12290,4130,954,1185,2794


In [363]:
data_family.loc['percentage'] = (data_family.loc['total'] / data_family.loc[1]).astype(int)
data_family
#посчитали процентное отношение для удобства сравнения результатов

family_status_id,0,1,2,3,4
debt,,,,,
0,11363,3744,892,1100,2521
1,927,386,62,85,273
total,12290,4130,954,1185,2794
percentage,13,10,15,13,10


In [364]:
data_children = data_sort.pivot_table(index=['debt'], columns='children', values='total_income', aggfunc='count')
data_children
#следует привести данные к целочисленным
#убрать пропущенные значения, привести к 0 
data_children.fillna(0, inplace=True)
data_children = data_children.loc[0:1].astype(int)


In [365]:
data_children.loc['total'] = data_children.loc[0] + data_children.loc[1]
data_children
data_children.loc['persentage'] = round(data_children.loc['total'] / data_children.loc[1], 2)
data_children 
#посчитали общее количество данных по каждому статусу
#посчитали процентное отношение для удобства сравнения результатов

children,0,1,2,3,4,5
debt,,,,,,
0,12964.00,4397.00,1912.00,301.00,37.00,9.0
1,1058.00,442.00,202.00,27.00,4.00,0.0
total,14022.00,4839.00,2114.00,328.00,41.00,9.0
persentage,13.25,10.95,10.47,12.15,10.25,inf


Для того, чтобы исследоват столбец с доходом нам следует его разделить на категории. Для этого узнаем минимальное и максимальное значение столбцов в каждой возрастной категории. воможно следует скорректировать таблицу и  выделить группу 1 - взрослые и 0 - пенсионеры

In [366]:
data_sort['total_income'].min()
#Cледует классифицировать данные по age group и доходу

20667

In [367]:
data_sort['total_income'].max()

2265604

In [368]:
data_sort['total_income'].median()

156025.0

In [369]:
def data_income(row):
    income = row['total_income']
    if income < 50000:
            return '0'
    if income <= 150000:
            return '1'
    if income <= 350000:
            return '2'
    if income <= 500000:
            return '3'
    return '4'
data_sort['income_category'] = data_sort.apply(data_income, axis = 1)
#мы создали столбец с новыми значениями датафрейма, где разделили клиентов на категории
#0 - низкий уровень дохода
#1 - cредний
#2 - выше среднего
#3 - высокий


<ipython-input-369-d28178a35d70>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sort['income_category'] = data_sort.apply(data_income, axis = 1)


In [370]:
data_sort.head()

,children,education_id,family_status_id,debt,total_income,credit_purpose,age_group,income_category
0,1,0,0,0,253875,недвижимость,взрослые,2
1,1,1,0,0,112080,автомобиль,взрослые,1
2,0,1,0,0,145885,недвижимость,взрослые,1
3,3,1,0,0,267628,образование,взрослые,2
4,0,1,1,0,158616,свадьба,взрослые,2


In [371]:
data_income = data_sort.pivot_table(index=['debt'], columns='income_category', values='total_income', aggfunc='count')
data_income


income_category,0,1,2,3,4
debt,,,,,
0,347,8934,9537,594,208
1,23,818,837,41,14


In [372]:
data_income.loc['total'] = data_income.loc[0] + data_income.loc[1]
data_income


income_category,0,1,2,3,4
debt,,,,,
0,347,8934,9537,594,208
1,23,818,837,41,14
total,370,9752,10374,635,222


In [373]:
data_income.loc['persentage'] = round(data_income.loc['total'] / data_income.loc[1], 2)
data_income

income_category,0,1,2,3,4
debt,,,,,
0,347.00,8934.00,9537.00,594.00,208.00
1,23.00,818.00,837.00,41.00,14.00
total,370.00,9752.00,10374.00,635.00,222.00
persentage,16.09,11.92,12.39,15.49,15.86


Решение на вопрос: Как разные цели кредита влияют на его возврат в срок?


In [374]:
data_debt = data[data['debt'] == 1]

data_debt.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,credit_purpose,age_group
14,0,1844,56,высшее,0,гражданский брак,1,f,компаньон,1,165127,покупка жилой недвижимости,"[покупка, , жилой, , недвижимость, \n]",недвижимость,взрослые
32,0,4649,34,среднее,1,гражданский брак,1,f,сотрудник,1,139057,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,взрослые
38,0,597,25,высшее,0,не женат / не замужем,4,m,сотрудник,1,192247,образование,"[образование, \n]",образование,взрослые
55,0,66914,54,среднее,1,гражданский брак,1,f,пенсионер,1,167465,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,взрослые
75,1,2953,38,среднее,1,женат / замужем,0,m,сотрудник,1,81935,операции с недвижимостью,"[операция, , с, , недвижимость, \n]",недвижимость,взрослые


In [375]:
round((data_sort['credit_purpose'] == 'недвижимость').sum() / (data_debt['credit_purpose'] == 'недвижимость').sum(), 2)


13.82

In [376]:
round((data_sort['credit_purpose'] == 'автомобиль').sum() / (data_debt['credit_purpose'] == 'автомобиль').sum(), 2)

10.71

In [377]:
(data_sort['credit_purpose'] == 'образование').sum() / (data_debt['credit_purpose'] == 'образование').sum()

10.797297297297296

In [378]:
round((data_sort['credit_purpose'] == 'свадьба').sum() / (data_debt['credit_purpose'] == 'свадьба').sum(), 2)

12.55

Вывод, мы получили результаты необходимые для ответа на поставленные вопросы. Составили сводные таблицы Data_family, data_children, data_income. Где наглядно можно увидеть процент, показывающий как семейное положение, наличие детей или доход влияют на выплату кредита в срок.
 C помощью простой математической операции провели исследование причин кридита и узнали процент , показывающий как цель кредита влияет на его выплату вовремя. 

# Шаг 3

Вывод: Есть ли зависимость между наличием детей и возвратом кредита в срок?

в среднем люди, которые имеют от 1 до 5 детей возвращают кредит так же, 
как и клиенты без детей. Напротив, процент наличия задолженности у клиентов без детей на 2% выше и составляет 13% 
в отличие от остальных, чей процент едва переходит порог в 10%. 
Это средний показатель, а значит нет, скорее не влияет 

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Исследование показывает, что самый высокий процент невозврата кредита в срок имеют те, кто потерял супруга. 
Их процент составляет 15%.
Наличие пары или холостой образ жизни имеют одинаковый процент 10-13%. 
Это средний показатель, а значит нет, скорее не влияет. 

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Возврат кредита в срок зависит от уровня дохода.
Самый большой процент имеет категория людей с низким доходом, он составляет 16%, но стоит отметить,
что люди имеющие высокий доход (более 350 000 ежемесячный доход) так же имеют высокие показатели невозврата кредита в срок,
результаты стремительно приближаются к отметке в 16%. 


Как разные цели кредита влияют на его возврат в срок?
В ходе исследования было выявлено, в данных представлены 4 главных цели кредита:

Если цель кредита недвижимость, то вовремя не возвращают кредит 13%
Если цель кредита свадьба, то вовремя не возвращают кредит 12%
Люди, которые берут кредит на автомобиль, имеют процент невозврата кредита в срок 10%
Кредит на образование так же не возвращают вовремя 10-11%


Общий вывод:
    В ходе анализа было выявлено несколько ошибок в данных. Для точности исследования они были исправлены. 
1. На этапе предобработки все данные приведены к нижнему регистру для дальнейшего удобства их использования.
2. Исправлены отрицательные значения на положительные в графе «общий трудовой стаж»
3. В графе «общий трудовой стаж» заполнены отсутствующие значения на средний по столбцу.
4. В графе «возраст» найдена ошибка в возрасте, где указано, что некоторым заемщикам меньше 18 лет. Так как таких значений было немного, чтобы они не мешали дальнейшему исследованию было принято решение удалить 101 заемщика. 
5. В графе «дети» так же была выявлена ошибка в виде отрицательных значений. Они были исправлены на положительные. А количество детей «20» я посчитала опечаткой и исправила значения на 2. 
6. Так же пропущенные значения встречаются в графе total_income (ежемесячный доход) в таком же количестве у тех же людей, что и пропуски в «трудовом стаже работы», скорее это тоже ошибка технического характера, так как сферв занятости семейный статус, возраст клиентов разный и нет видимых причин, почему пропуски могли бы быть оставлены специально. Пропуски были заменены на средние по столбцу. Я не удалила этих клиентов, потому что они составляют примерно 10%от всех клиентов. 
7. В ходе исследования было выявлено 54 дубликата в данных, было принято решение удалить дубликаты, скорее всего это сбой в программе. 
8. В предоставленных данных не было общей формулировки целей кредита, мною был создан отдельный столбец, где я привела все данные под категории «недвижимость», «автомобиль», «свадьба», «образование».
9. В ходе исследования было замечено, что некоторые категории взрослых людей в возрасте до 60 имеют статус «пенсионеры». Стоит уточнить у сотрудников банка по какой причине стоит данный статус. На всякий случай я категоризировала данные и создала новый столбец по возрасту, где отмечено, что люди до 60 взрослые, а пенсионеры после 60. 
10. Для того, чтобы ответить на поставленные вопросы, я создала сводные таблицы, где в процентном соотношении указано, какие категории людей не возвращают долг вовремя. В среднем на возврат кредита не влияет количество детей в семье, семейный статус или цель. Большая зависимость в невозврате от дохода. 
11. В 16% случаев имеют долги по кредитам те, у кого доход ниже 40 000 или выше 350 000.
12. Данные были категоризированы и приведены в числовым значениям для удобства исследования. Дополнительную информацию можно найти в выводах или спросить у меня. 


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.
- [x ]  открыт файл;
- [ x]  файл изучен;
- [ x]  определены пропущенные значения;
- [ x]  заполнены пропущенные значения;
- [ x]  есть пояснение какие пропущенные значения обнаружены;
- [ x]  описаны возможные причины появления пропусков в данных;
- [ x]  объяснено по какому принципу заполнены пропуски;
- [ x]  заменен вещественный тип данных на целочисленный;
- [ x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [ х]  удалены дубликаты;
- [ х]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [ х]  описаны возможные причины появления дубликатов в данных;
- [ х]  выделены леммы в значениях столбца с целями получения кредита;
- [ х]  описан процесс лемматизации;
- [ х]  данные категоризированы;
- [ х]  есть объяснение принципа категоризации данных;
- [ х]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ х]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ х]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ х]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [ х]  в каждом этапе есть выводы;
- [ х]  есть общий вывод.